In [45]:
import requests
import bs4
import os
import sympy

class EquationScraper():
    def __init__(self, doi, api_key):
        self.doi = doi
        self.api_key = api_key

    def make_request(self):
        response = requests.get(f"https://api.elsevier.com/content/article/doi/{self.doi}?APIKey={self.api_key}")
        if response.status_code == 200:
            print("Request successful \n")
            self.full_text = response.text 
        else:
            raise Exception(f"Request failed with status code: {response.status_code} \n")

    def make_soup(self):
        self.soup = bs4.BeautifulSoup(self.full_text, "lxml")

    def find_equations(self):
        self.equation = self.soup.find_all("ce:formula")
        equation_dict = {}
        for eq in self.equation:
            label = eq.find("ce:label")
            cleaned_label = int(label.text[1:-1])
            equation_dict[cleaned_label] = eq.find("mml:math")
        self.mathml_dict = equation_dict
    
    def mathml_to_python(self, mathml):
        pass
        
        

        
doi = "10.1016/j.triboint.2021.106934"
api_key = os.getenv("ELSEVIER_API_KEY")
scraper = EquationScraper(doi, api_key)
scraper.make_request()
scraper.make_soup()
scraper.find_equations()
print(scraper.mathml_dict[7])
print(len(scraper.mathml_dict))

Request successful 

<mml:math altimg="si34.svg" alttext="Equation 7.">
<mml:mrow>
<mml:mi>Δ</mml:mi>
<mml:mi>V</mml:mi>
<mml:mo linebreak="badbreak">=</mml:mo>
<mml:munderover>
<mml:mrow>
<mml:mo>∫</mml:mo>
</mml:mrow>
<mml:mn>0</mml:mn>
<mml:mi>t</mml:mi>
</mml:munderover>
<mml:mi>A</mml:mi>
<mml:mo linebreak="goodbreak">⋅</mml:mo>
<mml:mi>b</mml:mi>
<mml:mfrac>
<mml:mrow>
<mml:msup>
<mml:mi>P</mml:mi>
<mml:msub>
<mml:mi>k</mml:mi>
<mml:mn>3</mml:mn>
</mml:msub>
</mml:msup>
<mml:msup>
<mml:mi>v</mml:mi>
<mml:msub>
<mml:mi>k</mml:mi>
<mml:mn>1</mml:mn>
</mml:msub>
</mml:msup>
</mml:mrow>
<mml:msup>
<mml:mi>η</mml:mi>
<mml:msub>
<mml:mi>k</mml:mi>
<mml:mn>2</mml:mn>
</mml:msub>
</mml:msup>
</mml:mfrac>
<mml:mo linebreak="goodbreak">⋅</mml:mo>
<mml:mi>h</mml:mi>
<mml:mrow>
<mml:mo stretchy="false">(</mml:mo>
<mml:mi>t</mml:mi>
<mml:mo stretchy="false">)</mml:mo>
</mml:mrow>
<mml:mi>d</mml:mi>
<mml:mi>t</mml:mi>
</mml:mrow>
</mml:math>
10


In [ ]:
"""<ce:formula id="eq0010">
<ce:label>(1)</ce:label>
<mml:math altimg="si4.gif" overflow="scroll">
<mml:mrow>
<mml:mi>h</mml:mi>
<mml:mo>=</mml:mo>
<mml:msub>
<mml:mi>h</mml:mi>
<mml:mi>g</mml:mi>
</mml:msub>
<mml:mo>+</mml:mo>
<mml:msub>
<mml:mi>h</mml:mi>
<mml:mi>c</mml:mi>
</mml:msub>
</mml:mrow>
</mml:math>
</ce:formula>"""